**Libraries**

In [162]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense 
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.utils import to_categorical
import tensorflow as tf
from sklearn.metrics import accuracy_score
from keras import callbacks
from keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [163]:
alex = pd.read_excel('data/lab6_train_data.xlsx', sheet_name='Alex')
kelly = pd.read_excel('data/lab6_train_data.xlsx', sheet_name='Kelly')
test = pd.read_excel('data/lab6_train_data.xlsx', sheet_name='Test')

In [164]:
import glob
from PIL import Image
import numpy as np
import pandas as pd

def load_and_flatten_images(root_dir,
                            exts=(".jpg", ".jpeg", ".png", ".bmp"),
                            target_size=(64, 64)):
    """
    1) Walks through `root_dir`, finds all files ending in `exts` (recursively).
    2) Opens each image, converts to RGB, resizes to `target_size`.
    3) Flattens the H×W×C array into a single vector of length (H*W*C).
    4) Returns: a pandas DataFrame where each row is one image, with:
         • column "filename" (the basename of the image file), and
         • columns "pixel_0", "pixel_1", …, "pixel_{(H*W*C)-1}".
    """
    all_files = []
    for ext in exts:
        all_files.extend(glob.glob(f"{root_dir}/**/*{ext}", recursive=True))

    rows = []
    for fp in all_files:
        try:
            img = Image.open(fp).convert("RGB")
        except Exception as e:
            print(f"✗ Could not open {fp}: {e}")
            continue

        if target_size is not None:
            img = img.resize(target_size, resample=Image.BILINEAR)

        arr = np.array(img)  # shape = (H, W, 3)
        flat = arr.reshape(-1)  # shape = (H*W*3,)

        # build a single row: [filename, pixel_0, pixel_1, ..., pixel_n]
        basename = fp.split("/")[-1]  # or os.path.basename(fp)
        row = [basename] + flat.tolist()
        rows.append(row)

    if len(rows) == 0:
        return pd.DataFrame()  # empty

    # build column names: first is "filename", then "pixel_0", …, "pixel_{num_pixels-1}"
    num_pixels = target_size[0] * target_size[1] * 3
    col_names = ["Image ID"] + [f"pixel_{i}" for i in range(num_pixels)]
    df = pd.DataFrame(rows, columns=col_names)
    return df


In [165]:
#Alex Images
folder_path = "Alex_Kelly_Pics/Alex"
alex_images = load_and_flatten_images(folder_path, target_size=(64, 64))
print(alex_images.shape)         # e.g., (N_images, 1 + 224*224*3)
print(alex_images.columns[:5])  # ['filename', 'pixel_0', 'pixel_1', 'pixel_2', 'pixel_3']
print(alex_images.iloc[0, :10]) # show first image's name + first few pixel values

# If you really need a NumPy array for “everything but the filename”:
X = alex_images.drop(columns=["Image ID"]).to_numpy()   # shape = (N_images, 224*224*3)
filenames = alex_images["Image ID"].tolist()

(256, 12289)
Index(['Image ID', 'pixel_0', 'pixel_1', 'pixel_2', 'pixel_3'], dtype='object')
Image ID    Alex-Image119.png
pixel_0                    88
pixel_1                   103
pixel_2                   118
pixel_3                    91
pixel_4                   108
pixel_5                   122
pixel_6                    94
pixel_7                   111
pixel_8                   126
Name: 0, dtype: object
(256, 12289)
Index(['Image ID', 'pixel_0', 'pixel_1', 'pixel_2', 'pixel_3'], dtype='object')
Image ID    Alex-Image119.png
pixel_0                    88
pixel_1                   103
pixel_2                   118
pixel_3                    91
pixel_4                   108
pixel_5                   122
pixel_6                    94
pixel_7                   111
pixel_8                   126
Name: 0, dtype: object


In [166]:
#Kelly Images
folder_path = "Alex_Kelly_Pics/Kelly"
kelly_images = load_and_flatten_images(folder_path, target_size=(64, 64))
print(kelly_images.shape)         # e.g., (N_images, 1 + 224*224*3)
print(kelly_images.columns[:5])  # ['filename', 'pixel_0', 'pixel_1', 'pixel_2', 'pixel_3']
print(kelly_images.iloc[0, :10]) # show first image's name + first few pixel values

# If you really need a NumPy array for “everything but the filename”:
X = kelly_images.drop(columns=["Image ID"]).to_numpy()   # shape = (N_images, 224*224*3)
filenames = kelly_images["Image ID"].tolist()

(229, 12289)
Index(['Image ID', 'pixel_0', 'pixel_1', 'pixel_2', 'pixel_3'], dtype='object')
Image ID    Kelly-Image124.png
pixel_0                     91
pixel_1                     87
pixel_2                     81
pixel_3                     92
pixel_4                     88
pixel_5                     81
pixel_6                     89
pixel_7                     85
pixel_8                     77
Name: 0, dtype: object
(229, 12289)
Index(['Image ID', 'pixel_0', 'pixel_1', 'pixel_2', 'pixel_3'], dtype='object')
Image ID    Kelly-Image124.png
pixel_0                     91
pixel_1                     87
pixel_2                     81
pixel_3                     92
pixel_4                     88
pixel_5                     81
pixel_6                     89
pixel_7                     85
pixel_8                     77
Name: 0, dtype: object


In [167]:
#TestSet Images
folder_path = "Alex_Kelly_Pics/TestSet"
test_images = load_and_flatten_images(folder_path, target_size=(64, 64))
print(test_images.shape)         # e.g., (N_images, 1 + 224*224*3)
print(test_images.columns[:5])  # ['filename', 'pixel_0', 'pixel_1', 'pixel_2', 'pixel_3']
print(test_images.iloc[0, :10]) # show first image's name + first few pixel values

# If you really need a NumPy array for “everything but the filename”:
X = test_images.drop(columns=["Image ID"]).to_numpy()   # shape = (N_images, 224*224*3)
filenames = test_images["Image ID"].tolist()

(20, 12289)
Index(['Image ID', 'pixel_0', 'pixel_1', 'pixel_2', 'pixel_3'], dtype='object')
Image ID    TestSetImage01.png
pixel_0                     93
pixel_1                     88
pixel_2                     76
pixel_3                     59
pixel_4                     58
pixel_5                     56
pixel_6                     62
pixel_7                     63
pixel_8                     69
Name: 0, dtype: object
(20, 12289)
Index(['Image ID', 'pixel_0', 'pixel_1', 'pixel_2', 'pixel_3'], dtype='object')
Image ID    TestSetImage01.png
pixel_0                     93
pixel_1                     88
pixel_2                     76
pixel_3                     59
pixel_4                     58
pixel_5                     56
pixel_6                     62
pixel_7                     63
pixel_8                     69
Name: 0, dtype: object


In [168]:
#combine into single image
#df["filename"].str.removesuffix(".png")


In [169]:
#stack cat df
train = pd.concat([alex, kelly], ignore_index=True)

#stack images df
train_images = pd.concat([alex_images, kelly_images], ignore_index=True)

#remove .png to merge datasets
train_images["Image ID"] = train_images["Image ID"].str.removesuffix(".png")
test_images["Image ID"] = test_images["Image ID"].str.removesuffix(".png")

#merge data
train = pd.merge(train_images, train, on="Image ID")
test = pd.merge(test_images, test, on="Image ID")

**Modeling**

Approach 1: 

* Train a (convolutional) neural network to identify whether or not there is a human in the image.
* Train a (convolutional) neural network to identify whether or not there is a castle in the image.
* Train a (convolutional) neural network to identify whether the image is taken indoors or outdoors.
* Train a (convolutional) neural network to identify the landscape of the image (e.g. city, suburb, or nature/rural)
* Choose at least 10 other features (or feature categories) that you suspect might be useful for differentiating Alex and Kelly's photos, and train individual (convolutional) neural networks to identify those.

In [170]:
model = models.Sequential()

#input layer
model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(64, 64, 3)))

#hidden layers
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(64, 64, 3)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.25))

#output layer
model.add(layers.Dense(1, activation='sigmoid'))

#summarize
model.summary()

/Users/benlaufer/Library/Python/3.9/lib/python/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_8"
Model: "sequential_8"


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_16 (Conv2D)              │ (None, 64, 64, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_8 (Flatten)             │ (None, 262144)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 128)            │    33,554,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤

 Total params: 33,574,081 (128.07 MB)
 Total params: 33,574,081 (128.07 MB)


 Trainable params: 33,574,081 (128.07 MB)
 Trainable params: 33,574,081 (128.07 MB)


 Non-trainable params: 0 (0.00 B)
 Non-trainable params: 0 (0.00 B)


In [171]:
#compile model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


**Train a (convolutional) neural network to identify whether or not there is a human in the image.**

In [172]:
#variable selection
X = train.drop(columns=['Image ID', 'Human', 'Castle', 'Indoors or Outdoors', 'Landscape (City, Suburb, or Nature/Rural)', 'Child/Baby', 'Animal', 'Cat', 'Dog', 'Body_of_Water', 'Car', 'Bridge', 'Food', 'Tree', 'Mountain', 'Instrument', 'Drink'])
y = train['Human']

In [173]:
X = X.to_numpy().reshape(-1, 64, 64, 3)

In [174]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [175]:
# early call back
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min')

#fit model
history = model.fit(
    X, y,
    epochs=50,
    batch_size=64,
    callbacks=[early_stopping],
    verbose=0
)


/Users/benlaufer/Library/Python/3.9/lib/python/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


/Users/benlaufer/Library/Python/3.9/lib/python/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


/Users/benlaufer/Library/Python/3.9/lib/python/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


/Users/benlaufer/Library/Python/3.9/lib/python/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


/Users/benlaufer/Library/Python/3.9/lib/python/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


In [176]:
#get predictions
y_pred = model.predict(X_test)
#not sure what this does
y_pred = [np.argmax(i) for i in y_pred]


1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step

3/4 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step


In [177]:
#get accuracy score
accuracy_score(y_test, y_pred)

0.711340206185567

**Train a (convolutional) neural network to identify whether or not there is a castle in the image.**

In [178]:
#variable selection
X = train.drop(columns=['Image ID', 'Human', 'Castle', 'Indoors or Outdoors', 'Landscape (City, Suburb, or Nature/Rural)', 'Child/Baby', 'Animal', 'Cat', 'Dog', 'Body_of_Water', 'Car', 'Bridge', 'Food', 'Tree', 'Mountain', 'Instrument', 'Drink'])
y = train['Castle']

In [179]:
X = X.to_numpy().reshape(-1, 64, 64, 3)

In [180]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [181]:
# early call back
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min')

#fit model
history = model.fit(
    X, y,
    epochs=50,
    batch_size=64,
    callbacks=[early_stopping],
    verbose=0
)


/Users/benlaufer/Library/Python/3.9/lib/python/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


In [182]:
#get predictions
y_pred = model.predict(X_test)
#not sure what this does
y_pred = [np.argmax(i) for i in y_pred]


1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step

3/4 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


In [183]:
#get accuracy score
accuracy_score(y_test, y_pred)

0.9381443298969072

**Train a (convolutional) neural network to identify whether the image is taken indoors or outdoors.**

In [184]:
#variable selection
X = train.drop(columns=['Image ID', 'Human', 'Castle', 'Indoors or Outdoors', 'Landscape (City, Suburb, or Nature/Rural)', 'Child/Baby', 'Animal', 'Cat', 'Dog', 'Body_of_Water', 'Car', 'Bridge', 'Food', 'Tree', 'Mountain', 'Instrument', 'Drink'])
y = train['Indoors or Outdoors']

In [185]:
X = X.to_numpy().reshape(-1, 64, 64, 3)

In [186]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [187]:
# early call back
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min')

#fit model
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=64,
    callbacks=[early_stopping],
    verbose=0
)


/Users/benlaufer/Library/Python/3.9/lib/python/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


In [188]:
#get predictions
y_pred = model.predict(X_test)
#not sure what this does
y_pred = [np.argmax(i) for i in y_pred]


1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step

3/4 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step


In [189]:
#get accuracy score
accuracy_score(y_test, y_pred)

0.7422680412371134

**Choose at least 10 other features (or feature categories) that you suspect might be useful for differentiating Alex and Kelly's photos, and train individual (convolutional) neural networks to identify those.**

**Child/Baby**

In [190]:
#variable selection
X = train.drop(columns=['Image ID', 'Human', 'Castle', 'Indoors or Outdoors', 'Landscape (City, Suburb, or Nature/Rural)', 'Child/Baby', 'Animal', 'Cat', 'Dog', 'Body_of_Water', 'Car', 'Bridge', 'Food', 'Tree', 'Mountain', 'Instrument', 'Drink'])
y = train['Child/Baby']

In [191]:
X = X.to_numpy().reshape(-1, 64, 64, 3)

In [192]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [193]:
# early call back
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min')

#fit model
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=64,
    callbacks=[early_stopping],
    verbose=0
)


/Users/benlaufer/Library/Python/3.9/lib/python/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


In [194]:
#get predictions
y_pred = model.predict(X_test)
#not sure what this does
y_pred = [np.argmax(i) for i in y_pred]


1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step

3/4 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step


In [195]:
#get accuracy score
accuracy_score(y_test, y_pred)

0.9484536082474226

**Animal**

In [196]:
#variable selection
X = train.drop(columns=['Image ID', 'Human', 'Castle', 'Indoors or Outdoors', 'Landscape (City, Suburb, or Nature/Rural)', 'Child/Baby', 'Animal', 'Cat', 'Dog', 'Body_of_Water', 'Car', 'Bridge', 'Food', 'Tree', 'Mountain', 'Instrument', 'Drink'])
y = train['Animal']

In [197]:
X = X.to_numpy().reshape(-1, 64, 64, 3)

In [198]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [199]:
# early call back
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min')

#fit model
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=64,
    callbacks=[early_stopping],
    verbose=0
)


/Users/benlaufer/Library/Python/3.9/lib/python/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


KeyboardInterrupt: 

In [ ]:
#get predictions
y_pred = model.predict(X_test)
#not sure what this does
y_pred = [np.argmax(i) for i in y_pred]


In [200]:
#get accuracy score
accuracy_score(y_test, y_pred)

0.9484536082474226

**Cat**

In [201]:
#variable selection
X = train.drop(columns=['Image ID', 'Human', 'Castle', 'Indoors or Outdoors', 'Landscape (City, Suburb, or Nature/Rural)', 'Child/Baby', 'Animal', 'Cat', 'Dog', 'Body_of_Water', 'Car', 'Bridge', 'Food', 'Tree', 'Mountain', 'Instrument', 'Drink'])
y = train['Cat']

In [202]:
X = X.to_numpy().reshape(-1, 64, 64, 3)

In [203]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [204]:
# early call back
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min')

#fit model
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=64,
    callbacks=[early_stopping],
    verbose=0
)


KeyboardInterrupt: 

In [ ]:
#get predictions
y_pred = model.predict(X_test)
#not sure what this does
y_pred = [np.argmax(i) for i in y_pred]


In [205]:
#get accuracy score
accuracy_score(y_test, y_pred)

0.979381443298969

**Need to finish these variables too**

**Dog**

**Body_of_water**

**Car**

**Bridge**

**Food**

**Tree**

**Mountain**

**Instrument**

**Drink**

**Train a (convolutional) neural network to identify the landscape of the image (e.g. city, suburb, or nature/rural)**

In [206]:
model = models.Sequential()

#input layer
model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(64, 64, 3)))

#hidden layers
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(64, 64, 3)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.25))

#output layer
model.add(layers.Dense(3, activation='softmax'))

#summarize
model.summary()

Model: "sequential_9"
Model: "sequential_9"


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_18 (Conv2D)              │ (None, 64, 64, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_9 (Flatten)             │ (None, 262144)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 128)            │    33,554,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤

 Total params: 33,574,339 (128.08 MB)
 Total params: 33,574,339 (128.08 MB)


 Trainable params: 33,574,339 (128.08 MB)
 Trainable params: 33,574,339 (128.08 MB)


 Non-trainable params: 0 (0.00 B)
 Non-trainable params: 0 (0.00 B)


/Users/benlaufer/Library/Python/3.9/lib/python/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [207]:
#compile model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [208]:
#variable selection
X = train.drop(columns=['Image ID', 'Human', 'Castle', 'Indoors or Outdoors', 'Landscape (City, Suburb, or Nature/Rural)', 'Child/Baby', 'Animal', 'Cat', 'Dog', 'Body_of_Water', 'Car', 'Bridge', 'Food', 'Tree', 'Mountain', 'Instrument', 'Drink'])
y = train['Landscape (City, Suburb, or Nature/Rural)']

In [209]:
#reshape X
X = X.to_numpy().reshape(-1, 64, 64, 3)

In [210]:
#encode y categories and one-hot encode

encoder = LabelEncoder()
y_int = encoder.fit_transform(y)

#y_cat = keras.utils.to_categorical(y_int, num_classes=3)

#no class imbalance with landscape type, so there is no need to use special scoring metrics

In [211]:
X_train, X_test, y_train_int, y_test_int = train_test_split(
    X, y_int, test_size=0.2, random_state=42, stratify=y_int
)

In [212]:
print(X_train.shape)       # should be (X, 64, 64, 3)
print(y_train_int.shape)   # should be (X,)
print(X_test.shape)        # should be (y, 64, 64, 3)
print(y_test_int.shape)    # should be (y,)

(388, 64, 64, 3)
(388,)
(97, 64, 64, 3)
(97,)
(388, 64, 64, 3)
(388,)
(97, 64, 64, 3)
(97,)


In [213]:
y_train_cat = to_categorical(y_train_int, num_classes=3)
# Now y_train_cat.shape should be (388, 3)
print(y_train_cat.shape)

(388, 3)
(388, 3)


In [214]:
# early call back
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min')

#fit model
history = model.fit(
    X_train, y_train_cat,
    epochs=50,
    batch_size=64,
    callbacks=[early_stopping],
    verbose=0
)


KeyboardInterrupt: 

In [ ]:
#get predictions
y_pred = model.predict(X_test)
#not sure what this does
y_pred = [np.argmax(i) for i in y_pred]


In [215]:
#get accuracy score
accuracy_score(y_test_int, y_pred)

0.32989690721649484